### Neuromaps Correlation Analysis on Status Epilepticus

**Author**: Joseph Turner

##### Module imports and data loading

The following code block imports the necessary modules and loads the data from the Neuromaps database. 

We are fetching two annotations from the Neuromaps database that are cerebral blood flow maps (see [here](https://docs.google.com/spreadsheets/d/1oZecOsvtQEh5pQkIf8cB6CyhPKVrQuko/))

The first annotation is in surface space, and the second is in volume space. Later on we will compare the two annotations to our volumetric status epilepticus map. 

In [30]:
from neuromaps.datasets import available_annotations, fetch_annotation, fetch_mni152
from neuromaps.resampling import resample_images
from neuromaps.stats import compare_images
from neuromaps import transforms
import nibabel as nib

# The first annotation (raichle) is in fsLR space at 164k vertices
annotation1 = ('raichle', 'cbf', 'fsLR', '164k')
an1 = fetch_annotation(source=annotation1[0],  data_dir='data')

# The second annotation (satterthwaite2014) is in MNI152 space at 1mm resolution
annotation2 = ('satterthwaite2014', 'meancbf', 'MNI152', '1mm')
an2 = fetch_annotation(source=annotation2[0], data_dir='data')

# The status epilepticus map is in MNI152 space at 2mm resolution
img_path = 'se_maps/status_epilepticus.nii'
img = nib.load(img_path)


**Compare to the first blood flow annotation from Raichle**

In [36]:
img_transformed = transforms.mni152_to_fslr(img_path, fslr_density='164k')
img_transformed[0].to_filename('se_maps/status_epilepticus_fsL.gii')
img_transformed[1].to_filename('se_maps/status_epilepticus_fsR.gii')


r1 = compare_images(src='se_maps/status_epilepticus_fsL.gii', trg='data/annotations/raichle/cbf/fsLR/source-raichle_desc-cbf_space-fsLR_den-164k_hemi-L_feature.func.gii', metric='pearsonr')
r2 = compare_images(src='se_mapsstatus_epilepticus_fsR.gii', trg='data/annotations/raichle/cbf/fsLR/source-raichle_desc-cbf_space-fsLR_den-164k_hemi-R_feature.func.gii', metric='pearsonr')

print(f"Correlation between status epilepticus and raichle (fsLR, 164k, L): {r1:.4f}")
print(f"Correlation between status epilepticus and raichle (fsLR, 164k, R): {r2:.4f}")

Correlation between status epilepticus and raichle (fsLR, 164k, L): 0.0998
Correlation between status epilepticus and raichle (fsLR, 164k, R): -0.0118


**Compare to the second blood flow annotation from Satterthwaite et al. 2014**

In [37]:
an2_transformed = transforms.mni152_to_mni152(an2, target=img_path)
squeezed_data = an2_transformed.get_fdata().squeeze()
an2_transformed = nib.Nifti1Image(squeezed_data, an2_transformed.affine)
an2_transformed.shape, img.shape
type(an2_transformed), type(img)

r = compare_images(src=img.get_fdata(), trg=an2_transformed.get_fdata(), metric='pearsonr')
print(f"Correlation between status epilepticus and satterthwaite2014: {r:.4f}")


Correlation between status epilepticus and satterthwaite2014: -0.0318
